## Day 22: Sporifica Virus 

Diagnostics indicate that the local **grid computing cluster** has been contaminated with the **Sporifica Virus**. The grid computing cluster is a seemingly-infinite two-dimensional grid of compute nodes. Each node is either **clean** or **infected** by the virus.

### Part One

To [prevent overloading](https://en.wikipedia.org/wiki/Morris_worm#The_mistake) the nodes (which would render them useless to the virus) or detection by system administrators, exactly one virus carrier moves through the network, infecting or cleaning nodes as it moves. The virus carrier is always located on a single node in the network (the current node) and keeps track of the direction it is facing.

In [3]:
from enum import Enum
# Reused form Day 19

class Direction(Enum):

    NORTH = 1
    EAST = 2
    SOUTH = 4
    WEST = 8

    def __neg__(self):  # So -NORTH == SOUTH. Pretty cool!
        map_oposites = {
            Direction.NORTH: Direction.SOUTH,
            Direction.SOUTH: Direction.NORTH,
            Direction.EAST: Direction.WEST,
            Direction.WEST: Direction.EAST,
            }
        return map_oposites[self]
    
    def __str__(self):
        if self == Direction.NORTH:
            return '▲'
        elif self == Direction.SOUTH:
            return '▼'
        elif self == Direction.WEST:
            return '◄'
        else:
            return '►'
    
def rotate_left(direction):
    return {
        Direction.NORTH: Direction.WEST,
        Direction.WEST: Direction.SOUTH,
        Direction.SOUTH: Direction.EAST,
        Direction.EAST: Direction.NORTH,
    }.get(direction)

# tests
assert rotate_left(Direction.NORTH) == Direction.WEST
assert rotate_left(Direction.WEST) == Direction.SOUTH
assert rotate_left(Direction.SOUTH) == Direction.EAST
assert rotate_left(Direction.EAST) == Direction.NORTH
for d in Direction:
    assert rotate_left(rotate_left(rotate_left(rotate_left(d)))) == d

def rotate_right(direction):
    return {
        Direction.NORTH: Direction.EAST,
        Direction.EAST: Direction.SOUTH,
        Direction.SOUTH: Direction.WEST,
        Direction.WEST: Direction.NORTH,
    }.get(direction)

# tests
assert rotate_right(Direction.NORTH) == Direction.EAST
assert rotate_right(Direction.EAST) == Direction.SOUTH
assert rotate_right(Direction.SOUTH) == Direction.WEST
assert rotate_right(Direction.WEST) == Direction.NORTH
for d in Direction:
    assert rotate_right(rotate_right(rotate_right(rotate_right(d)))) == d

In [5]:
class Virus:
    
    def __init__(self):
        self.x = 0
        self.y = 0
        self.direction = Direction.NORTH
        
    def __str__(self):
        return "Virus(x={}, y={}, direction='{}')".format(
            self.x,
            self.y,
            self.direction,
            )
    
    def move(self):
        if self.direction == Direction.NORTH:
            self.y -= 1
        elif self.direction == Direction.WEST:
            self.x -= 1
        elif self.direction == Direction.SOUTH:
            self.y += 1
        elif self.direction == Direction.EAST:
            self.x += 1
            
        
v = Virus()
print(v)
v.move()
assert v.y == -1
assert v.x == 0

Virus(x=0, y=0, direction='▲')


To avoid detection, the virus carrier works in bursts; in each burst, it wakes up, does some work, and goes back to sleep. The following steps are all executed in order one time each burst:

 - If the **.current node** is infected, it turns to its **right**. Otherwise, it turns to its **left** (Turning is done in-place; the **current node** does not change.)
 
 - If the **current node** is clean, it becomes **infected**. Otherwise, it becomes cleaned. (This is done after the node is considered for the purposes of changing direction.)
 
 - The virus carrier **moves** forward one node in the direction it is facing.

In [9]:
def burst(grid, virus):
    infected = grid.get(virus.x, virus.y)
    if infected:
        virus.direction = rotate_right(virus.direction)
    else:
        virus.direction = rotate_left(virus.direction)
    grid.put(virus.x, virus.y, not infected)
    virus.move()
    return not infected

> We will need a Grid map:

In [33]:
from collections import defaultdict

class Grid:
    
    def __init__(self, default_function=bool):
        self.kernel = defaultdict(default_function)
        self.bound_min_x = 0
        self.bound_max_x = 0
        self.bound_min_y = 0
        self.bound_max_y = 0
        
    def recalc_bounds(self, x, y):
        self.bound_min_x = min(x, self.bound_min_x)
        self.bound_max_x = max(x, self.bound_max_x)
        self.bound_min_y = min(y, self.bound_min_y)
        self.bound_max_y = max(y, self.bound_max_y)
        
    def get(self, x, y):
        self.recalc_bounds(x, y)
        pos = (x, y)
        return self.kernel[pos]
    
    def put(self, x, y, value):
        self.recalc_bounds(x, y)
        pos = (x, y)
        self.kernel[pos] = value
        
    def __str__(self):
        rows = []
        for y in range(self.bound_min_y, self.bound_max_y+1):
            cols = []
            for x in range(self.bound_min_x, self.bound_max_x+1):
                if x == 0 and y == 0:
                    cols.append('◈' if self.kernel[x,y] else '◇')
                else:
                    cols.append('▣' if self.kernel[x,y] else '▢')
            rows.append(''.join(cols))
        return '\n'.join(rows)
            
    
grid = Grid()
grid.get(0, 0) == False
assert grid.get(-2, -2) == False
grid.put(1,1, True)
print(grid)


▢▢▢▢
▢▢▢▢
▢▢◇▢
▢▢▢▣


Diagnostics have also provided a map of the node infection status (your puzzle input). Clean nodes are shown as .; infected nodes are shown as #. This map only shows the center of the grid; there are many more nodes beyond those shown, but none of them are currently infected.

In [11]:
def load_grid(filename):
    grid = Grid()
    with open(filename, 'r') as f:
        lines = [l.strip() for l in f]
    offset_x = len(lines[0]) // 2
    offset_y = len(lines) // 2
    for y, line in enumerate(lines):
        for x, c in enumerate(list(line)):
            grid.put(x - offset_x, y - offset_y, c == '#')
    return grid
    
grid = load_grid('input.txt')
print(grid)

▣▣▢▣▣▣▢▢▢▢▢▣▣▢▢▣▢▣▣▣▣▢▢▢▢
▣▣▢▢▢▣▢▣▢▣▢▢▣▣▢▣▢▢▢▢▣▢▣▢▢
▢▢▢▣▢▢▣▢▣▣▣▢▣▢▣▣▣▢▣▣▢▣▣▣▣
▢▢▣▣▢▢▣▣▣▢▢▢▢▣▢▣▣▢▣▢▢▣▣▢▣
▣▣▣▢▢▢▢▣▣▣▣▣▢▢▣▣▣▢▣▢▢▣▢▢▣
▢▢▢▢▢▣▢▣▢▢▢▣▢▢▣▣▢▢▣▢▣▣▢▢▢
▢▣▣▢▣▢▣▣▣▢▣▢▣▢▢▢▣▣▢▣▢▣▣▢▣
▢▢▢▢▢▢▣▣▣▣▣▣▢▣▣▣▢▢▢▢▢▢▣▣▣
▣▢▢▢▢▢▣▣▢▣▢▢▢▢▣▢▢▢▣▢▢▢▢▢▢
▢▢▢▢▣▢▢▣▣▣▢▣▢▣▢▣▣▣▣▢▣▣▢▣▢
▢▣▢▣▢▣▣▢▢▢▣▣▣▢▣▣▣▣▣▣▢▣▣▣▣
▣▣▣▣▢▢▢▢▢▢▣▢▢▢▣▢▢▢▣▢▢▣▢▣▢
▣▣▣▢▣▣▢▣▣▢▢▣◈▢▢▢▢▣▢▢▣▣▢▣▢
▢▢▣▢▣▣▣▢▣▣▢▢▣▢▢▢▣▣▣▣▣▣▣▢▢
▢▢▢▣▣▣▣▢▣▢▢▢▣▣▣▢▢▣▢▢▣▣▣▢▣
▢▢▣▢▣▢▢▢▢▢▢▢▣▢▣▣▣▣▢▣▢▢▢▢▢
▢▢▣▣▢▢▣▣▣▣▢▣▣▣▣▣▣▢▢▣▣▢▣▣▣
▢▢▣▢▢▣▢▢▣▣▢▢▢▣▢▣▣▣▣▢▢▢▢▣▢
▢▣▢▢▣▢▣▣▣▣▢▣▢▢▣▣▢▢▣▢▢▣▣▢▢
▢▢▢▢▢▢▣▣▣▣▣▢▢▢▣▢▣▣▢▣▢▢▢▢▣
▣▣▣▢▢▣▢▢▢▣▢▣▢▢▢▣▢▣▢▢▣▢▣▢▣
▢▣▢▣▣▣▢▣▢▢▢▢▣▣▢▢▣▣▣▣▣▣▢▣▣
▣▣▢▣▣▣▣▣▣▢▢▢▢▢▣▣▢▣▢▣▢▣▢▢▣
▢▢▣▢▢▣▣▢▣▣▢▢▣▢▣▢▢▣▣▣▢▣▣▢▢
▣▢▣▣▢▣▣▢▢▣▣▢▣▢▣▣▣▢▢▢▢▢▢▢▣


The virus carrier begins in the middle of the map facing up.

For example, suppose you are given a map like this:

    ..#
    #..
    ...

Then, the middle of the infinite grid looks like this, with the virus carrier's position marked with [ ]:

    . . . . . . . . .
    . . . . . . . . .
    . . . . . . . . .
    . . . . . # . . .
    . . . #[.]. . . .
    . . . . . . . . .
    . . . . . . . . .
    . . . . . . . . .

The virus carrier is on a clean node, so it turns left, infects the node, and moves left:

    . . . . . . . . .
    . . . . . . . . .
    . . . . . . . . .
    . . . . . # . . .
    . . .[#]# . . . .
    . . . . . . . . .
    . . . . . . . . .
    . . . . . . . . .

The virus carrier is on an infected node, so it turns right, cleans the node, and moves up:

    . . . . . . . . .
    . . . . . . . . .
    . . . . . . . . .
    . . .[.]. # . . .
    . . . . # . . . .
    . . . . . . . . .
    . . . . . . . . .
    . . . . . . . . .

Four times in a row, the virus carrier finds a clean, infects it, turns left, and moves forward, ending in the same place and still facing up:

    . . . . . . . . .
    . . . . . . . . .
    . . . . . . . . .
    . . #[#]. # . . .
    . . # # # . . . .
    . . . . . . . . .
    . . . . . . . . .
    . . . . . . . . .

Now on the same node as before, it sees an infection, which causes it to turn right, clean the node, and move forward:

    . . . . . . . . .
    . . . . . . . . .
    . . . . . . . . .
    . . # .[.]# . . .
    . . # # # . . . .
    . . . . . . . . .
    . . . . . . . . .
    . . . . . . . . .

After the above actions, a total of 7 bursts of activity had taken place. Of them, 5 bursts of activity caused an infection.

After a total of 70, the grid looks like this, with the virus carrier facing up:

    . . . . . # # . .
    . . . . # . . # .
    . . . # . . . . #
    . . # . #[.]. . #
    . . # . # . . # .
    . . . . . # # . .
    . . . . . . . . .
    . . . . . . . . .

In [12]:
grid = load_grid('input_test.txt')
virus = Virus()
count = 0
for _ in range(70):
    count += burst(grid, virus)
print(grid)
print(count)

▢▢▢▣▣▢▢
▢▢▣▢▢▣▢
▢▣▢▢▢▢▣
▣▢▣▢▢▢▣
▣▢◈▢▢▣▢
▢▢▢▣▣▢▢
41


By this time, 41 bursts of activity caused an infection (though most of those nodes have since been cleaned).

After a total of 10000 bursts of activity, 5587 bursts will have caused an infection.

In [13]:
grid = load_grid('input_test.txt')
virus = Virus()
count = 0
for _ in range(10000):
    count += burst(grid, virus)
print('Part one:', count)

Part one: 5587


Given your actual map, **after 10000 bursts of activity, how many bursts cause a node to become infected**? (Do not count nodes that begin infected.)

In [15]:
grid = load_grid('input.txt')
virus = Virus()
count = 0
for _ in range(10000):
    count += burst(grid, virus)
print('Part one:', count)

Part one: 5552


### Part Two

As you go to remove the virus from the infected nodes, it evolves to resist your attempt.

Now, before it infects a clean node, it will weaken it to disable your defenses. If it encounters an infected node, it will instead flag the node to be cleaned in the future. So:

- Clean nodes become weakened.
- Weakened nodes become infected.
- Infected nodes become flagged.
- Flagged nodes become clean.

Every node is always in exactly one of the above states.

In [70]:
from enum import Enum

class State(Enum):
    
    CLEAN = 1
    WEAKENED = 2
    INFECTED = 4
    FLAGGED = 8
    
    def next(self):
        return {
            State.CLEAN: State.WEAKENED,
            State.WEAKENED: State.INFECTED,
            State.INFECTED: State.FLAGGED,
            State.FLAGGED: State.CLEAN,
        }.get(self)
    
    def __str__(self):
        if self == State.CLEAN:
            return '·'
        elif self == State.WEAKENED:
            return 'W'
        elif self == State.INFECTED:
            return '▩'
        else:
            return 'F'
        
    
# tests

assert State.CLEAN.next() == State.WEAKENED
assert State.WEAKENED.next() == State.INFECTED
assert State.INFECTED.next() == State.FLAGGED
assert State.FLAGGED.next() == State.CLEAN
for state in State:
    state.next().next().next().next == state
print('ok')

def clean_state():
    return State.CLEAN

def home(state):
    if state == State.CLEAN:
        return 'Ⓒ'
    elif state == State.WEAKENED:
        return 'Ⓦ'
    elif state == State.INFECTED:
        return 'Ⓘ'
    else:
        return 'Ⓕ'
    

    
class GridTwo(Grid):
            
    def __str__(self):
        rows = []
        for y in range(self.bound_min_y, self.bound_max_y+1):
            cols = []
            for x in range(self.bound_min_x, self.bound_max_x+1):
                if x == 0 and y == 0:
                    cols.append(home(self.kernel[x,y]))
                else:
                    cols.append(str(self.kernel[x,y]))
            rows.append(''.join(cols))
        return '\n'.join(rows)
    
grid = GridTwo(clean_state)
grid.put(-2, -1, State.WEAKENED)
grid.put(1, 2, State.FLAGGED)
grid.put(-2, 2, State.INFECTED)
print(grid)

ok
W···
··Ⓒ·
····
▩··F


In [91]:
def load_grid_two(filename):
    grid = GridTwo(clean_state)
    with open(filename, 'r') as f:
        lines = [l.strip() for l in f]
    offset_x = len(lines[0]) // 2
    offset_y = len(lines) // 2
    for y, line in enumerate(lines):
        for x, c in enumerate(list(line)):
            grid.put(x - offset_x, y - offset_y, State.INFECTED if c == '#' else State.CLEAN)
    return grid

The virus carrier still functions in a similar way, but now uses the following logic during its bursts of action:

 - Decide which way to turn based on the current node:
    - If it is clean, it turns left.
    - If it is weakened, it does not turn, and will continue moving in the same direction.
    - If it is infected, it turns right.
    - If it is flagged, it reverses direction, and will go back the way it came.
 - Modify the state of the current node, as described above.
 - The virus carrier moves forward one node in the direction it is facing.

In [92]:
def burst_two(grid, virus):
    s = grid.get(virus.x, virus.y)
    if s == State.CLEAN:
        virus.direction = rotate_left(virus.direction)
    elif s == State.WEAKENED:
        pass
    elif s == State.INFECTED:
        virus.direction = rotate_right(virus.direction)
    else:
        virus.direction = - virus.direction
    grid.put(virus.x, virus.y, s.next())
    virus.move()
    return 1 if s == State.WEAKENED else 0


Start with the same map (still using . for clean and # for infected) and still with the virus carrier starting in the middle and facing up.

Using the same initial state as the previous example, and drawing weakened as W and flagged as F, the middle of the infinite grid looks like this, with the virus carrier's position again marked with [ ]:

    . . . . . . . . .
    . . . . . . . . .
    . . . . . . . . .
    . . . . . # . . .
    . . . #[.]. . . .
    . . . . . . . . .
    . . . . . . . . .
    . . . . . . . . .

In [103]:
grid = load_grid_two('input_test.txt')
virus = Virus()
print(grid)

··▩
▩Ⓒ·
···


This is the same as before, since no initial nodes are weakened or flagged. The virus carrier is on a clean node, so it still turns left, instead weakens the node, and moves left:

    . . . . . . . . .
    . . . . . . . . .
    . . . . . . . . .
    . . . . . # . . .
    . . .[#]W . . . .
    . . . . . . . . .
    . . . . . . . . .
    . . . . . . . . .

In [104]:
burst_two(grid, virus)
print(grid)

··▩
▩Ⓦ·
···


The virus carrier is on an infected node, so it still turns right, instead flags the node, and moves up:

    . . . . . . . . .
    . . . . . . . . .
    . . . . . . . . .
    . . .[.]. # . . .
    . . . F W . . . .
    . . . . . . . . .
    . . . . . . . . .
    . . . . . . . . .

In [105]:
burst_two(grid, virus)
print(grid)

··▩
FⓌ·
···


This process repeats three more times, ending on the previously-flagged node and facing right:

    . . . . . . . . .
    . . . . . . . . .
    . . . . . . . . .
    . . W W . # . . .
    . . W[F]W . . . .
    . . . . . . . . .
    . . . . . . . . .
    . . . . . . . . .

In [106]:
for _ in range(3):
    burst_two(grid, virus)
print(grid)


WW·▩
WFⓌ·
····


Finding a flagged node, it reverses direction and cleans the node:

    . . . . . . . . .
    . . . . . . . . .
    . . . . . . . . .
    . . W W . # . . .
    . .[W]. W . . . .
    . . . . . . . . .
    . . . . . . . . .
    . . . . . . . . .

In [107]:
burst_two(grid, virus)
print(grid)

WW·▩
W·Ⓦ·
····


The weakened node becomes infected, and it continues in the same direction:

    . . . . . . . . .
    . . . . . . . . .
    . . . . . . . . .
    . . W W . # . . .
    .[.]# . W . . . .
    . . . . . . . . .
    . . . . . . . . .
    . . . . . . . . .
    

In [108]:
burst_two(grid, virus)
print(grid)

WW·▩
▩·Ⓦ·
····


Of the first 100 bursts, 26 will result in infection. Unfortunately, another feature of this evolved virus is speed; of the first 10000000 bursts, 2511944 will result in infection.

In [109]:
grid = load_grid_two('input_test.txt')
virus = Virus()
counter = 0
for _ in range(100):
    counter += burst_two(grid, virus)
print(counter)

26


In [110]:
grid = load_grid_two('input_test.txt')
virus = Virus()
counter = 0
for _ in range(10000000):
    counter += burst_two(grid, virus)
print(counter)

2511944


Given your actual map, **after 10000000 bursts of activity, how many bursts cause a node to become infected?** (Do not count nodes that begin infected.)

In [111]:
grid = load_grid_two('input.txt')
virus = Virus()
counter = 0
for _ in range(10000000):
    if _ % 100000 == 0:
        print('#', end='')
    counter += burst_two(grid, virus)
print()
print(counter)

####################################################################################################
2511527
